# **Conoce el conjunto de datos**

Es fundamental comprender la estructura del dataset y el significado de sus columnas.

In [33]:
#Importación de las librerías necesarias
import pandas as pd
import os

In [34]:
# Definimos la ruta al archivo limpio
ruta_limpia = os.path.join("..", "data", "processed", "telecom_final.csv")

try:
    df = pd.read_csv(ruta_limpia)
    print(f"Datos cargados exitosamente: {df.shape[0]} filas, {df.shape[1]} columnas.")
except FileNotFoundError:
    print("Error: No se encuentra el archivo. Ejecuta primero el notebook 1.0.")

Datos cargados exitosamente: 7267 filas, 21 columnas.


In [35]:
#Vamos a verificar si los datos se cargaron correctamente
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7043 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


In [36]:
# Verificamos que 'Total', 'Monthly' y 'tenure' sean numéricos 
display(df.dtypes)

customerID           object
Churn                object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
Monthly             float64
Total               float64
dtype: object

##  Diccionario de Datos & Identificación de Variables

| Variable | Descripción | Tipo | ¿Relevante para Churn? |
|----------|-------------|------|------------------------|
| **customerID** | ID único del cliente | Texto |  No (Es solo un identificador) |
| **gender** | Género (Male/Female) | Texto |  Quizás (Demográfico) |
| **SeniorCitizen**| ¿Es jubilado? (1,0) | Entero |  Sí (Perfil de riesgo) |
| **Partner/Dependents** | Familia | Texto |  Sí (Estabilidad) |
| **tenure** | Meses que lleva en la empresa | Entero |  **Muy Crítico** (Fidelidad) |
| **Phone/InternetService** | Servicios contratados | Texto |  Sí (Calidad de servicio) |
| **Contract** | Tipo de contrato (Mes, Año) | Texto |  **Muy Crítico** (Barrera de salida) |
| **Charges.Monthly** | Factura mensual | Float |  **Muy Crítico** (Sensibilidad precio) |
| **Total** | Total pagado en la historia | Float |  Sí (Valor del cliente/LTV) |
| **Churn** | **Variable Objetivo** (Se fue o no) | Texto |  **TARGET** |

In [37]:
# Lista de columnas que identificamos como CRÍTICAS para el negocio
cols_relevantes = ['Contract', 'tenure', 'Monthly', 'Total', 'Churn']

display(df[cols_relevantes].head(10))

,Contract,tenure,Monthly,Total,Churn
0,One year,9,65.60,593.30,No
1,Month-to-month,9,59.90,542.40,No
2,Month-to-month,4,73.90,280.85,Yes
3,Month-to-month,13,98.00,1237.85,Yes
4,Month-to-month,3,83.90,267.40,Yes
5,Month-to-month,9,69.40,571.45,No
6,Two year,71,109.70,7904.25,No
7,Two year,63,84.65,5377.80,No
8,Month-to-month,7,48.20,340.35,No
9,Two year,65,90.45,5957.90,No


## Comprobación de incoherencias

In [38]:
cols_texto = df.select_dtypes(include=['object']).columns

for col in cols_texto:
    if col != 'customerID':
        print(f"valores en '{col}': {df[col].unique()}")


valores en 'Churn': ['No' 'Yes' nan]
valores en 'gender': ['Female' 'Male']
valores en 'Partner': ['Yes' 'No']
valores en 'Dependents': ['Yes' 'No']
valores en 'PhoneService': ['Yes' 'No']
valores en 'MultipleLines': ['No' 'Yes' 'No phone service']
valores en 'InternetService': ['DSL' 'Fiber optic' 'No']
valores en 'OnlineSecurity': ['No' 'Yes' 'No internet service']
valores en 'OnlineBackup': ['Yes' 'No' 'No internet service']
valores en 'DeviceProtection': ['No' 'Yes' 'No internet service']
valores en 'TechSupport': ['Yes' 'No' 'No internet service']
valores en 'StreamingTV': ['Yes' 'No' 'No internet service']
valores en 'StreamingMovies': ['No' 'Yes' 'No internet service']
valores en 'Contract': ['One year' 'Month-to-month' 'Two year']
valores en 'PaperlessBilling': ['Yes' 'No']
valores en 'PaymentMethod': ['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']


In [39]:
# Verificamos si hay IDs repetidos
duplicados = df.duplicated(subset=['customerID']).sum()
print(f"Clientes duplicados: {duplicados}")

Clientes duplicados: 0


In [40]:
nulos = df.isnull().sum()
print(nulos[nulos > 0])

Churn    224
dtype: int64


In [41]:
clientes_ilogicos = df[(df['tenure'] == 0) & (df['Total'] > 0)]
print(f"Clientes con datos ilógicos (Tenure=0 pero Total>0): {len(clientes_ilogicos)}")

Clientes con datos ilógicos (Tenure=0 pero Total>0): 0


## Ajustar datos para que estén completos y coherentes

In [42]:
if df['Total'].isnull().sum() > 0:
    df['Total'] = df['Total'].fillna(0)
    print("\nValores nulos en 'Total' rellenados con 0.")


In [43]:
if duplicados > 0:
    df.drop_duplicates(subset=['customerID'], keep='first', inplace=True)
    print("Duplicados eliminados.")

In [44]:
# Si hubiera clientes ilógicos, los corregimos.
if len(clientes_ilogicos) > 0:
    df.loc[df['tenure'] == 0, 'Total'] = 0
    print("Corregidos clientes con antigüedad 0 y cobros erróneos.")


In [45]:
cols_a_convertir = ['gender', 'Partner', 'Dependents', 'PhoneService', 
                    'InternetService', 'Contract', 'Churn']

for col in cols_a_convertir:
    if col in df.columns:
        df[col] = df[col].astype('category')

display(df.dtypes)


customerID            object
Churn               category
gender              category
SeniorCitizen          int64
Partner             category
Dependents          category
tenure                 int64
PhoneService        category
MultipleLines         object
InternetService     category
OnlineSecurity        object
OnlineBackup          object
DeviceProtection      object
TechSupport           object
StreamingTV           object
StreamingMovies       object
Contract            category
PaperlessBilling      object
PaymentMethod         object
Monthly              float64
Total                float64
dtype: object

## Columna de cuentas diarias 

In [46]:
df['Cuentas_Diarias'] = df['Monthly'] / 30

In [47]:
display(df[['Monthly', 'Cuentas_Diarias']].head())

,Monthly,Cuentas_Diarias
0,65.6,2.186667
1,59.9,1.996667
2,73.9,2.463333
3,98.0,3.266667
4,83.9,2.796667


In [48]:
display(df['Cuentas_Diarias'].describe().round(2))

count    7267.00
mean        2.16
std         1.00
min         0.61
25%         1.18
50%         2.34
75%         3.00
max         3.96
Name: Cuentas_Diarias, dtype: float64